### **przykład**

- konwersja etykiety do wartości całkowitej: od 0 do liczba_klas-1 przy pomocy LabelEncoder
- wczytać plik label_to_num.csv, zawierający jedną kolumnę ‘Frogs’, a w niej sześć różnych gatunków

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
dataset = pd.read_csv('label_to_num.csv', header=0)
print(dataset)

In [ ]:
label_encoder = LabelEncoder()
dataset['Frogs'] = label_encoder.fit_transform(dataset['Frogs'])
print(dataset)

### **zadanie 1**

klasyfikacja z wykorzystaniem algorytmu drzew decyzyjnych:
- wykonanie zadania klasyfikacyjnego z zastosowaniem algorytmu drzew decyzyjnych
- dwa sposoby wizualizacji utworzonego drzewa
- interpretacja modelu dostępnego w formie graficznej (zaleta DT)
- analiza wyniku walidacji, finalna ocena zdolności predykcyjnych modelu klasyfikacyjnego, ale także analiza utworzonego drzewa, które dostarcza wielu informacji o rozwiązywanym problemie
- zapis drzewa w różnym formacie

In [ ]:
!pip install graphviz

In [ ]:
!pip install -q dtreeviz

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import graphviz
import dtreeviz

In [ ]:
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv -O /tmp/penguins.csv
df = pd.read_csv("/tmp/penguins.csv")
df.head()

In [ ]:
df = df.dropna() #usuwanie wartości NaN
df.head()

In [ ]:
label_encoder = LabelEncoder()
df['island'] = label_encoder.fit_transform(df['island'])
df.head()

In [ ]:
X = df.values[:, 1:6]
y = df.values[:, 0]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
clf = tree.DecisionTreeClassifier()
# definicja klasyfikatora
scores = cross_val_score(clf, X, y, cv=5)
# walidacja krzyżowa
print('Accuracy (mean): ', scores.mean())

In [ ]:
clf = clf.fit(X, y) # uczenie klasyfikatora
plt.figure(figsize=(10,7)) #odpowiednio dopasować: figsize, fontsize
tree.plot_tree(clf, fontsize=7, filled=True,
feature_names=df.columns[1:6], class_names=df["species"].unique())
plt.savefig('DTiris.png') #zapis do pliku
plt.show()
plt.show()

In [ ]:
dot_data = tree.export_graphviz(clf, # wyświetlenie drzewa decyzyjnego (model dla wszystkich przypadków)
feature_names=df.columns[1:6],
class_names=df["species"].unique(),
filled=True, rounded=True)
graph = graphviz.Source(dot_data)
graph #alteranatywnie: display(graph)
graph.format = "pdf"
graph.render("DT") #zapis do pliku

In [ ]:
viz = dtreeviz.model(clf, X, y,
target_name='spicies',
feature_names=df.columns[1:6],
class_names=df["species"].unique())
viz.view(scale=1.2, fontname="Liberation Mono")

**sugestie dalszych obliczeń:**
- zwrócić uwagę na wysokość drzewa oraz przeanalizować, które zmienne objaśniające zostały uwzględnione w modelu.
- skomentować jakość modelu na podstawie wykonanej walidacji krzyżowej.
- zwrócić uwagę na profesjonalną wizualizację drzew za pomocą bibliotek graphviz, dtreeviz.
- przeanalizować zawartość poszczególnych węzłów i liści DT.
- przetestować możliwość zapisu drzewa do pliku w innym formacie graficznym.
- przeanalizować, co oznacza informacja ‘gini’ w poszczególnych węzłach i liściach DT
- dodatkowo: sprawdzić wynik modelowania z pominięciem parametru ‘island’

### **zadanie 2**

klasyfikacja z wykorzystaniem algorytmu drzew decyzyjnych oraz przycinanie drzewa

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
import graphviz
import dtreeviz

In [ ]:
pima = pd.read_csv('diabetes.csv')
pima.head()

In [ ]:
# Krótkie nazwy cech (wygodne przy wyświetlaniu drzewa)
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
pima = pd.read_csv('diabetes.csv', header=0, names=col_names)
print(pima.head())

In [ ]:
X = pima.values[:, 0:8]  # cechy
y = pima.label          # etykieta
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
clf = DecisionTreeClassifier() # instancja klasyfikatora
clf = clf.fit(X_train,y_train) # trenowanie klasyfikatora
y_train_pred = clf.predict(X_train) # weryfikacja
print("Accuracy (train):",metrics.accuracy_score(y_train,y_train_pred))
y_test_pred = clf.predict(X_test)
print("Accuracy (test):",metrics.accuracy_score(y_test, y_test_pred))

In [ ]:
# wyświetlenie drzewa decyzyjnego (model dla wszystkich przypadków)
dot_data = tree.export_graphviz(clf,
                     feature_names=col_names[:-1],
                     class_names=['0','1'],
                     filled=True, rounded=True)
graph = graphviz.Source(dot_data, format='png')
graph.render('DTdiabetes') # do pliku DTdiabetes.png
graph

**uwagi i analiza wyników:**
- na tym etapie obliczeń uzyskujemy złożone DT. Bardzo duża liczba węzłów i głębokość drzewa sugerują, że model może być przeuczony. Ten fakt dyskwalifikuje wykorzystanie go do predykcji przynależności do klas nowych obiektów.
- optymalizujemy hiperparametry algorytmu (m.in. przycinanie drzewa):
clf = DecisionTreeClassifier(max_depth=?)

In [ ]:
clf = DecisionTreeClassifier(max_depth=5) # instancja klasyfikatora
clf = clf.fit(X_train,y_train) # trenowanie klasyfikatora
y_train_pred = clf.predict(X_train) # weryfikacja
print("Accuracy (train):",metrics.accuracy_score(y_train,y_train_pred))
y_test_pred = clf.predict(X_test)
print("Accuracy (test):",metrics.accuracy_score(y_test, y_test_pred))

In [ ]:
# wyświetlenie drzewa decyzyjnego (model dla wszystkich przypadków)
dot_data = tree.export_graphviz(clf,
                     feature_names=col_names[:-1],
                     class_names=['0','1'],
                     filled=True, rounded=True)
graph = graphviz.Source(dot_data, format='png')
graph.render('DTdiabetes') # do pliku DTdiabetes.png
graph

### **zadanie 3**

lasy losowe z badaniem istotności cech

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv -O /tmp/penguins.csv

df = pd.read_csv("/tmp/penguins.csv")
df.head()

In [ ]:
df = df.dropna() #usuwanie wartości NaN
df.head()

In [ ]:
label_encoder = LabelEncoder()
df['island'] = label_encoder.fit_transform(df['island'])
df.head()

In [ ]:
X = df.values[:, 1:6]
y = df.values[:, 0]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# obiekt klasyfikatora – liczba drzew i inne hiperparametry do optymalizacji
clf = RandomForestClassifier(n_estimators=10, random_state=10)
clf.fit(X_train, y_train) # trenowanie klasyfikatora
y_test_pred = clf.predict(X_test)
# dokładność dla zbioru testowego
print('\nAccuracy (test):', accuracy_score(y_test, y_test_pred))

In [ ]:
# Badanie istotności cech
print('\nFeature importances')
importances = pd.Series(clf.feature_importances_, index=df.columns[1:6])
print(importances)
importances.nlargest(5).plot(kind='barh')
plt.xlabel('Relative importance')
plt.show()

In [ ]:
# Macierz pomyłek i raport z klasyfikacji
print(confusion_matrix(y_test,y_test_pred))
print(classification_report(y_test,y_test_pred))

**sugestie:**
- modyfikować liczbę drzew w celu uzyskania wyższej jakości modelu.

In [ ]:
# obiekt klasyfikatora – liczba drzew i inne hiperparametry do optymalizacji
clf = RandomForestClassifier(n_estimators=35, random_state=10)
clf.fit(X_train, y_train) # trenowanie klasyfikatora
y_test_pred = clf.predict(X_test)
# dokładność dla zbioru testowego
print('\nAccuracy (test):', accuracy_score(y_test, y_test_pred))

In [ ]:
# Badanie istotności cech
print('\nFeature importances')
importances = pd.Series(clf.feature_importances_, index=df.columns[1:6])
print(importances)
importances.nlargest(5).plot(kind='barh')
plt.xlabel('Relative importance')
plt.show()

In [ ]:
# Macierz pomyłek i raport z klasyfikacji
print(confusion_matrix(y_test,y_test_pred))
print(classification_report(y_test,y_test_pred))

**sugestie:**
- zdefiniować RF na podstawie 2-3 najbardziej istotnych cech. Jaki to ma wpływ na zdolność predykcyjną modelu?

In [ ]:
X = df.loc[:, ['bill_length_mm','flipper_length_mm','island']]

X = df.drop(['bill_depth_mm', 'body_mass_g', 'species', 'sex', 'year'], axis=1)

X= df.values[:,(1,2,4)]

y = df.values[:, 0]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# obiekt klasyfikatora – liczba drzew i inne hiperparametry do optymalizacji
clf = RandomForestClassifier(n_estimators=3)
clf.fit(X_train, y_train) # trenowanie klasyfikatora
y_test_pred = clf.predict(X_test)
# dokładność dla zbioru testowego
print('\nAccuracy (test):', accuracy_score(y_test, y_test_pred))

In [ ]:
# Macierz pomyłek i raport z klasyfikacji
print(confusion_matrix(y_test,y_test_pred))
print(classification_report(y_test,y_test_pred))

### **zadanie 4:**

lasy losowe z oceną modelu za pomocą AUC:
- w tym przykładzie optymalizujemy hiperparametry klasyfikatora
- dodatkowo ocena jakości jest realizowana za pomocą AUC uzyskanych podczas walidacji krzyżowej


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

In [ ]:
pima = pd.read_csv('diabetes.csv')
print(pima.head())
X = pima.values[:, 0:8]
y = pima.values[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
clf = RandomForestClassifier(n_estimators=10, random_state=10)
clf = clf.fit(X_train,y_train) # trenowanie
y_test_pred = clf.predict(X_test) # predykcja dla zbioru testowego
print("Accuracy (test):",metrics.accuracy_score(y_test, y_test_pred))

In [ ]:
# CV (cv=10), analiza AUC
score = cross_val_score(clf, X, y, cv=10, scoring='roc_auc')
print("Mean AUC score: ",score.mean())
print("Std AUC score: ",score.std())

In [ ]:
importances = pd.Series(clf.feature_importances_, index=pima.keys()[0:8])
print(importances)
importances.nlargest().plot(kind='barh')
plt.xlabel('Relative Importance')
plt.show()

**sugestie dalszych obliczeń:**
- ponieważ średnia wartość AUC wynosi znacznie mniej niż 1, podjąć działania mające na celu poprawę zdolności predykcyjnych.
- modyfikować hiperparametry klasyfikatora.
- wykorzystać cechy o najwyższym wskaźniku istotności w modelowaniu.

In [ ]:
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train,y_train) # trenowanie
y_test_pred = clf.predict(X_test) # predykcja dla zbioru testowego
print("Accuracy (test):",metrics.accuracy_score(y_test, y_test_pred))

In [ ]:
# CV (cv=10), analiza AUC
score = cross_val_score(clf, X, y, cv=6, scoring='roc_auc')
print("Mean AUC score: ",score.mean())
print("Std AUC score: ",score.std())

In [ ]:
importances = pd.Series(clf.feature_importances_, index=pima.keys()[0:8])
importances.nlargest().plot(kind='barh')
plt.xlabel('Relative Importance')
plt.show()